<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1


    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance


    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-18 12:23:25
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  6.17 L
Current:  1.43 C
-------------------
Today PnL: 29.79 K (0.21%)
Current PnL: -16.76 L (-11.27%)
CY Booked + Current PnL: -5.32 L (-3.58%)
-------------------
Total profit:  2.07 L
Total loss:  -18.83 L
-------------------
Total Booked + Current PnL: 21.66 L (17.8%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.01%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 82.12 L (57.49%)
Deployed:  1.22 C
Current:  1.43 C
CAGR/XIRR %: 9.93%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TTKPRESTIG,770.00,103.63,55.0,M-SC,3.98,87777.0,-13000.0,13096.0,-0.99,-12.90,14.92,0.09,245.0,-0.99,0.64,15.32,OX40N,NTT,DURABLES
56,RELIANCE,1533.00,-11.79,60.0,H-LC,5.18,220709.0,10343.0,18429.0,0.07,4.92,8.35,13.68,37.0,0.56,1.61,22.25,XY25,NTT,REFINERIES
49,NATIONALUM,247.44,-43.66,63.0,H-MC,6.80,112482.0,11978.0,18661.0,-0.48,11.92,16.59,30.49,79.0,0.64,0.82,48.18,MH,ATH,METALS
41,ITC,452.00,-37.92,51.0,H-LC,2.52,201150.0,1012.0,19874.0,0.50,0.51,9.88,10.44,4.0,0.05,1.47,6.10,X40,NTT,FMCG
48,MASFIN,398.61,-16.44,53.0,H-SC,12.99,94725.0,-3255.0,24856.0,-0.19,-3.32,26.24,22.05,152.0,-0.13,0.69,37.69,XR,ATH,FINANCE


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,43.54,62.0,M-MC,6.95,227149.0,2187.0,165478.0,-0.63,0.97,72.85,74.53,192.0,0.01,1.66,33.54,XY24,BTT,STEEL
35,IEX,219.00,-29.93,54.0,H-SC,17.09,200185.0,231.0,96990.0,-0.62,0.12,48.45,48.63,115.0,0.00,1.46,11.68,XR,NTT,MISC
29,HAVELLS,2069.16,-0.23,56.0,H-MC,1.86,247494.0,-237.0,75288.0,-0.44,-0.10,30.42,30.30,92.0,-0.00,1.81,13.18,X40,ATH,ELECTRICAL
51,PGHH,17973.08,-30.68,54.0,H-MC,4.20,200835.0,15.0,68766.0,0.04,0.01,34.24,34.25,80.0,0.00,1.47,5.21,X40,ATH,FMCG


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VBL,671.64,-17.33,39.0,H-LC,6.39,297978.0,-17864.0,130514.0,-0.85,-5.66,43.80,35.67,5.0,-0.14,2.18,7.41,X40N,ATH,FMCG
5,ASIANPAINT,3465.66,-13.06,46.0,H-LC,7.83,219349.0,-32419.0,85634.0,-0.04,-12.88,39.04,21.13,27.0,-0.38,1.60,17.99,X40,ATH,PAINTS
79,UNITDSPR,1644.00,-13.23,51.0,H-LC,7.81,227608.0,-6536.0,55149.0,-1.03,-2.79,24.23,20.77,86.0,-0.12,1.67,3.93,X40N,NTT,BREWERIES
41,ITC,452.00,-37.92,51.0,H-LC,2.52,201150.0,1012.0,19874.0,0.50,0.51,9.88,10.44,4.0,0.05,1.47,6.10,X40,NTT,FMCG
43,JIOFIN,387.00,0.39,53.0,H-LC,13.23,272792.0,5427.0,60042.0,0.43,2.03,22.01,24.48,48.0,0.09,2.00,58.12,XY24,BTT,FINANCE


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.2,0.87,72.0,H-LC,1.29,250290.0,4139.0,75062.0,2.56,1.68,29.99,32.18,16.0,0.06,1.83,38.33,X200,ATH,IT
41,ITC,452.0,-37.92,51.0,H-LC,2.52,201150.0,1012.0,19874.0,0.50,0.51,9.88,10.44,4.0,0.05,1.47,6.10,X40,NTT,FMCG
50,NESTLEIND,1377.0,-8.22,58.0,H-LC,3.86,282578.0,17152.0,39646.0,0.27,6.46,14.03,21.40,11.0,0.43,2.07,13.53,XY25,NTT,FMCG
56,RELIANCE,1533.0,-11.79,60.0,H-LC,5.18,220709.0,10343.0,18429.0,0.07,4.92,8.35,13.68,37.0,0.56,1.61,22.25,XY25,NTT,REFINERIES
19,CIPLA,1795.0,-19.44,54.0,H-LC,5.78,214843.0,10343.0,31066.0,0.55,5.06,14.46,20.25,10.0,0.33,1.57,14.23,X40N,BTT,PHARMA


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.20,0.87,72.0,H-LC,1.29,250290.0,4139.0,75062.0,2.56,1.68,29.99,32.18,16.0,0.06,1.83,38.33,X200,ATH,IT
60,SAMMAANCAP,326.00,-171.59,64.0,M-SC,2.65,82872.0,-19338.0,112731.0,2.26,-18.92,136.03,91.37,208.0,-0.17,0.61,34.53,XY25,NTT,FINANCE
7,ATULAUTO,844.00,3943.24,65.0,M-SC,5.80,126024.0,-21344.0,73157.0,1.98,-14.48,58.05,35.16,236.0,-0.29,0.92,28.72,XY24,NTT,AUTO
15,BSOFT,836.99,-12.08,57.0,H-SC,15.02,98927.0,-41726.0,114508.0,1.82,-29.67,115.75,51.74,131.0,-0.36,0.72,12.81,XR,ATH,IT
39,INFY,2275.00,-15.29,60.0,H-LC,8.39,330107.0,17219.0,154457.0,1.80,5.50,46.79,54.87,3.0,0.11,2.42,12.52,X40,BTT,IT


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,GREENPANEL,537.00,232.69,62.0,M-SC,2.45,148596.0,-34482.0,113453.0,-1.76,-18.83,76.35,43.14,230.0,-0.30,1.09,39.78,XY24,NTT,MISC
54,RAJOOENG,143.33,-36.44,37.0,H-SC,11.60,96420.0,-6080.0,46908.0,-1.61,-5.93,48.65,39.83,114.0,-0.13,0.71,0.78,AR,ATH,MISC
31,HINDZINC,730.22,20.84,58.0,M-LC,1.35,197142.0,-7934.0,120513.0,-1.36,-3.87,61.13,54.89,52.0,-0.07,1.44,19.29,X5K,ATH,METALS
69,SYMPHONY,1306.00,-42.09,39.0,M-SC,18.31,123507.0,-47634.0,47575.0,-1.10,-27.83,38.52,-0.03,196.0,-1.00,0.90,1.25,OX40N,NTT,DURABLES
3,ALKYLAMINE,4546.37,-4.34,48.0,H-SC,12.74,89053.0,-11910.0,106445.0,-1.09,-11.80,119.53,93.63,148.0,-0.11,0.65,32.64,SR,ATH,CHEMICALS


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TTKPRESTIG,770.0,103.63,55.0,M-SC,3.98,87777.0,-13000.0,13096.0,-0.99,-12.90,14.92,0.09,245.0,-0.99,0.64,15.32,OX40N,NTT,DURABLES
77,UJJIVANSFB,60.0,96.03,55.0,H-SC,9.53,123903.0,-18576.0,38100.0,0.31,-13.04,30.75,13.70,163.0,-0.49,0.91,46.24,OX40N,NTT,BANKS
44,KANSAINER,340.0,-67.06,64.0,H-SC,1.40,227259.0,-42408.0,78745.0,1.21,-15.73,34.65,13.47,138.0,-0.54,1.66,14.94,XY24,NTT,PAINTS
18,CERA,9475.0,-19.33,50.0,H-SC,2.08,146878.0,-29025.0,71045.0,0.19,-16.50,48.37,23.89,149.0,-0.41,1.07,26.85,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-19.68,54.0,H-MC,7.66,108756.0,-23079.0,65308.0,0.72,-17.51,60.05,32.03,98.0,-0.35,0.80,21.82,OX40N,NTT,IT


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4476.75,-25.15,59.0,H-LC,12.33,298262.0,-47694.0,122556.0,0.01,-13.79,41.09,21.64,1.0,-0.39,2.18,5.66,X40,ATH,IT
39,INFY,2275.00,-15.29,60.0,H-LC,8.39,330107.0,17219.0,154457.0,1.80,5.50,46.79,54.87,3.0,0.11,2.42,12.52,X40,BTT,IT
41,ITC,452.00,-37.92,51.0,H-LC,2.52,201150.0,1012.0,19874.0,0.50,0.51,9.88,10.44,4.0,0.05,1.47,6.10,X40,NTT,FMCG
82,VBL,671.64,-17.33,39.0,H-LC,6.39,297978.0,-17864.0,130514.0,-0.85,-5.66,43.80,35.67,5.0,-0.14,2.18,7.41,X40N,ATH,FMCG
1,ABB,7934.00,-37.73,67.0,H-LC,10.34,260438.0,-1181.0,120400.0,0.74,-0.45,46.23,45.57,7.0,-0.01,1.91,10.82,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,43.54,62.0,M-MC,6.95,227149.0,2187.0,165478.0,-0.63,0.97,72.85,74.53,192.0,0.01,1.66,33.54,XY24,BTT,STEEL
14,BLUESTARCO,2326.38,14.07,66.0,H-MC,3.58,197000.0,32330.0,35637.0,1.54,19.63,18.09,41.28,89.0,0.91,1.44,29.18,X40N,ATH,AC
83,VOLTAS,1918.49,1.86,61.0,H-MC,2.51,213600.0,21858.0,74183.0,0.86,11.40,34.73,50.08,99.0,0.29,1.56,18.97,XY25,ATH,AC
17,CAMS,5211.76,0.16,66.0,H-SC,3.82,113974.0,11970.0,31958.0,1.42,11.73,28.04,43.06,122.0,0.37,0.83,30.86,X40N,ATH,MISC
23,DABUR,735.00,-4.38,52.0,H-MC,6.70,208124.0,11096.0,78525.0,-0.28,5.63,37.73,45.49,102.0,0.14,1.52,19.41,XY24,BTT,FMCG


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,NATIONALUM,247.44,-43.66,63.0,H-MC,6.80,112482.0,11978.0,18661.0,-0.48,11.92,16.59,30.49,79.0,0.64,0.82,48.18,MH,ATH,METALS
84,WHIRLPOOL,2270.00,-40.79,50.0,M-SC,0.48,100050.0,8552.0,70195.0,-0.57,9.35,70.16,86.07,223.0,0.12,0.73,45.29,XR,NTT,DURABLES
36,INDIAMART,4850.92,-52.03,41.0,H-SC,11.66,133963.0,10627.0,123139.0,-0.91,8.62,91.92,108.45,119.0,0.09,0.98,32.90,AR,ATH,MISC
85,WIPRO,420.00,-9.26,60.0,M-LC,5.63,159266.0,8321.0,101134.0,1.07,5.51,63.50,72.51,53.0,0.08,1.17,11.87,XR,NTT,IT
11,BANKINDIA,190.00,-29.19,64.0,H-MC,6.11,183222.0,3414.0,108999.0,-0.43,1.90,59.49,62.52,88.0,0.03,1.34,36.63,XR,NTT,BANKS


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,RAJESHEXPO,518.00,1898.74,68.0,L-SC,3.04,54537.0,-82640.0,82733.0,0.47,-60.24,151.70,0.07,267.0,-1.00,0.40,35.64,OX40N,NTT,JEWELLERY
6,ASIANTILES,137.00,6700.00,52.0,L-SC,5.18,74615.0,-19195.0,95813.0,0.71,-20.46,128.41,81.67,269.0,-0.20,0.55,44.50,XR,NTT,CERAMICS
60,SAMMAANCAP,326.00,-171.59,64.0,M-SC,2.65,82872.0,-19338.0,112731.0,2.26,-18.92,136.03,91.37,208.0,-0.17,0.61,34.53,XY25,NTT,FINANCE
21,COFFEEDAY,80.00,-41.65,54.0,L-SC,42.44,86400.0,-27149.0,67202.0,1.51,-23.91,77.78,35.27,268.0,-0.40,0.63,111.47,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-4.34,48.0,H-SC,12.74,89053.0,-11910.0,106445.0,-1.09,-11.80,119.53,93.63,148.0,-0.11,0.65,32.64,SR,ATH,CHEMICALS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ATULAUTO,844.0,3943.24,65.0,M-SC,5.80,126024.0,-21344.0,73157.0,1.98,-14.48,58.05,35.16,236.0,-0.29,0.92,28.72,XY24,NTT,AUTO
21,COFFEEDAY,80.0,-41.65,54.0,L-SC,42.44,86400.0,-27149.0,67202.0,1.51,-23.91,77.78,35.27,268.0,-0.40,0.63,111.47,XR,NTT,HOTELS
60,SAMMAANCAP,326.0,-171.59,64.0,M-SC,2.65,82872.0,-19338.0,112731.0,2.26,-18.92,136.03,91.37,208.0,-0.17,0.61,34.53,XY25,NTT,FINANCE
80,VAIBHAVGBL,521.0,67.49,73.0,H-SC,7.21,145089.0,-37686.0,148760.0,1.61,-20.62,102.53,60.77,125.0,-0.25,1.06,29.95,XR,NTT,JEWELLERY
53,RAJESHEXPO,518.0,1898.74,68.0,L-SC,3.04,54537.0,-82640.0,82733.0,0.47,-60.24,151.70,0.07,267.0,-1.00,0.40,35.64,OX40N,NTT,JEWELLERY


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.11
1,25,43.86
2,50,74.45


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.54
LC    32.16
MC    23.27
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.27
X40      15.45
X40N     11.93
XR       11.77
AR        8.76
XY25      8.28
OX40N     8.23
X200      1.83
MH        1.71
X5K       1.44
SR        1.30
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.31
H-LC    25.44
H-MC    20.31
M-SC    13.65
M-LC     5.64
M-MC     2.65
L-SC     1.58
L-LC     1.08
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.71,-12.23,73.42
FMCG,11.87,-4.29,40.78
FINANCE,9.77,-15.18,60.30
MISC,7.48,-12.39,74.89
BANKS,6.32,-15.73,76.61
PAINTS,5.87,-14.93,32.06
ELECTRICAL,5.76,-4.55,62.16
AC,3.72,4.85,27.38
INSURANCE,3.45,-6.96,43.65


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2843402.0,22
XR,1301013.0,14
AR,1299987.0,9
X40,737080.0,10
X40N,597484.0,9
OX40N,521600.0,10
XY25,390758.0,6
SR,243928.0,2
X5K,120513.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3402649.0,29
M-SC,1380086.0,17
H-LC,1215662.0,14
H-MC,1176722.0,15
M-LC,379010.0,4
M-MC,315332.0,2
L-SC,245748.0,3
L-MC,60564.0,1
L-LC,36341.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1023413.0      6
M-SC       XY24       796444.0      7
H-SC       AR         787113.0      5
           XR         752624.0      7
H-MC       XY24       490808.0      4
H-LC       X40        453776.0      5
           AR         351978.0      2
M-MC       XY24       315332.0      2
H-SC       X40N       284600.0      4
           OX40N      248345.0      4
           SR         243928.0      2
H-LC       X40N       216729.0      3
H-MC       X40        213180.0      4
           XY25       183611.0      2
L-SC       XR         163015.0      2
M-SC       AR         160896.0      2
M-LC       XY24       157363.0      2
M-SC       OX40N      125214.0      4
M-LC       X5K        120513.0      1
M-SC       XR         114677.0      2
           XY25       112731.0      1
H-MC       XR         108999.0      1
M-LC       XR         101134.0      1
H-MC       X40N        96155.0      2
L-SC       OX40N       82733.0      1
H-LC       X200        75062.0      1
M-SC       X40         70124.0      1
H-MC       OX40N       65308.0      1
H-SC       MH          62626.0      1
L-MC       XR          60564.0      1
H-LC       XY24        60042.0      1
           XY25        58075.0      2
L-LC       XY25        36341.0      1
H-MC       MH          18661.0      1

In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 31.0 seconds
